# Tα2-3: Collapse → Ordered Sweep (3 ep/λ)

**Purpose**: Measure collapse-branch of hysteresis loop with slower sweep

**Protocol**:
- Load collapse checkpoint (from Tα-prep-C)
- Sweep λ: 0.70 → 0.30 (step 0.02, 21 points)
- Hold: 3 epochs per λ
- Record test error at each λ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_Ta2_collapse_sweep_3ep'
NOTEBOOK_ID = 'Ta2-3'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

# Checkpoint source directory
CHECKPOINT_DIR = sorted(glob.glob(f'{BASE_DIR}/exp_Ta_prep_collapse_*'))[-1] + '/checkpoints'
print(f'Checkpoint source: {CHECKPOINT_DIR}')

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Notebook: {NOTEBOOK_ID}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# Experiment parameters
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

# Sweep settings (DOWNWARD: collapse → ordered)
NOISE_RATE = 0.4
LAMBDA_START = 0.70
LAMBDA_END = 0.30
LAMBDA_STEP = 0.02
EPOCHS_PER_LAMBDA = 3  # Slower sweep

LAMBDA_GRID = np.arange(LAMBDA_START, LAMBDA_END - LAMBDA_STEP/2, -LAMBDA_STEP)
LAMBDA_GRID = np.round(LAMBDA_GRID, 2)

print(f'η = {NOISE_RATE}')
print(f'λ sweep: {LAMBDA_START} → {LAMBDA_END} (step {LAMBDA_STEP})')
print(f'λ points: {len(LAMBDA_GRID)} ({list(LAMBDA_GRID)})')
print(f'Epochs per λ: {EPOCHS_PER_LAMBDA}')
print(f'Total epochs per run: {len(LAMBDA_GRID) * EPOCHS_PER_LAMBDA}')

In [ ]:
# List available checkpoints
ckpt_files = sorted(glob.glob(f'{CHECKPOINT_DIR}/collapse_seed*.pth'))
print(f'Available collapse checkpoints: {len(ckpt_files)}')
for f in ckpt_files:
    print(f'  {os.path.basename(f)}')

# Select seeds to use (up to 5)
N_SEEDS = min(5, len(ckpt_files))
CHECKPOINT_FILES = ckpt_files[:N_SEEDS]
print(f'\nUsing {N_SEEDS} checkpoints')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_one_epoch(model, train_loader, opt, clean_t, noisy_t, lam, cached_gv_ref):
    crit = nn.CrossEntropyLoss()
    model.train()
    step = cached_gv_ref['step']
    cached_gv = cached_gv_ref['gv']
    
    for x, _, idx in train_loader:
        x, idx = x.to(device), idx.to(device)
        bn, bc = noisy_t[idx], clean_t[idx]
        
        opt.zero_grad()
        loss_s = crit(model(x), bn)
        loss_s.backward(retain_graph=True)
        gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        if step % K == 0 or cached_gv is None:
            opt.zero_grad()
            loss_v = crit(model(x), bc)
            loss_v.backward()
            cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        gs_n = gs / (gs.norm() + 1e-12)
        gv_n = cached_gv / (cached_gv.norm() + 1e-12)
        
        g_mix = (1 - lam) * gs_n + lam * gv_n
        opt.zero_grad()
        i = 0
        for p in model.parameters():
            n = p.numel()
            p.grad = g_mix[i:i+n].view(p.shape).clone()
            i += n
        opt.step()
        step += 1
    
    cached_gv_ref['step'] = step
    cached_gv_ref['gv'] = cached_gv

In [ ]:
def run_sweep(ckpt_path, train_loader, test_loader, clean_labels, noisy_labels):
    """Load collapse checkpoint and sweep λ downward"""
    
    # Load checkpoint
    ckpt = torch.load(ckpt_path, map_location=device)
    seed = ckpt['seed']
    init_error = ckpt['final_error']
    
    print(f'    Loaded checkpoint: seed={seed}, init_error={init_error:.4f}')
    
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    
    # Restore model
    set_seed(seed + 1000)
    model = get_resnet18().to(device)
    model.load_state_dict(ckpt['model_state_dict'])
    
    # Fresh optimizer for sweep
    opt = optim.SGD(model.parameters(), lr=LR * 0.01, momentum=0.9, weight_decay=5e-4)
    
    cached_gv_ref = {'step': 0, 'gv': None}
    trajectory = []
    
    print(f'    Sweeping λ: {LAMBDA_START} → {LAMBDA_END}...')
    
    for lam in LAMBDA_GRID:
        for _ in range(EPOCHS_PER_LAMBDA):
            train_one_epoch(model, train_loader, opt, clean_t, noisy_t, lam, cached_gv_ref)
        
        err = 1 - evaluate(model, test_loader)
        trajectory.append({'lambda': float(lam), 'error': err})
        print(f'      λ={lam:.2f}: err={err:.4f}')
    
    final_error = trajectory[-1]['error']
    
    return {
        'seed': seed,
        'init_error': init_error,
        'final_error': final_error,
        'lambda_start': LAMBDA_START,
        'lambda_end': LAMBDA_END,
        'epochs_per_lambda': EPOCHS_PER_LAMBDA,
        'trajectory': trajectory,
        'checkpoint_source': os.path.basename(ckpt_path)
    }

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

m = get_resnet18().to(device)
for _ in range(10): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt_file = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done_ckpts = set()

if os.path.exists(ckpt_file):
    results = json.load(open(ckpt_file))
    done_ckpts = {r['checkpoint_source'] for r in results}
    print(f'Loaded: {len(done_ckpts)} done')

for i, ckpt_path in enumerate(CHECKPOINT_FILES):
    ckpt_basename = os.path.basename(ckpt_path)
    if ckpt_basename in done_ckpts:
        continue
    
    run_num = len(results) + 1
    print(f'\n{"="*60}')
    print(f'[{run_num}/{N_SEEDS}] {ckpt_basename} | {NOTEBOOK_ID}')
    print(f'{"="*60}')
    
    ckpt_temp = torch.load(ckpt_path, map_location='cpu')
    seed = ckpt_temp['seed']
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    t0 = time.time()
    result = run_sweep(ckpt_path, train_loader, test_loader, clean_labels, noisy_labels)
    dt = time.time() - t0
    
    result['experiment_id'] = f'{NOTEBOOK_ID}-{run_num:03d}'
    result['eta'] = NOISE_RATE
    result['time'] = dt
    
    results.append(result)
    done_ckpts.add(ckpt_basename)
    
    print(f'\n  Init: {result["init_error"]:.2%} → Final: {result["final_error"]:.2%} | {dt/60:.1f}min')
    
    json.dump(results, open(ckpt_file, 'w'), indent=2, default=str)
    
    remaining = N_SEEDS - run_num
    print(f'  ETA: {remaining*dt/3600:.1f}h')
    
    torch.cuda.empty_cache()

print('\n' + '='*60)
print(f'{NOTEBOOK_ID} COMPLETE')
print('='*60)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

json.dump(results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2, default=str)

summary_data = []
for r in results:
    for t in r['trajectory']:
        summary_data.append({'seed': r['seed'], 'lambda': t['lambda'], 'error': t['error'], 'branch': 'collapse_down'})
df = pd.DataFrame(summary_data)
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print('='*60)
print(f'{NOTEBOOK_ID} SUMMARY')
print('='*60)
print(f'\nRuns: {len(results)}')
print(f'λ range: {LAMBDA_START} → {LAMBDA_END}')
print(f'Epochs per λ: {EPOCHS_PER_LAMBDA}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
for r in results:
    lams = [t['lambda'] for t in r['trajectory']]
    errs = [t['error'] for t in r['trajectory']]
    ax.plot(lams, errs, 's-', alpha=0.7, linewidth=2, markersize=4, label=f"seed {r['seed']}")
ax.axhline(0.40, color='red', linestyle='--', alpha=0.5, label='Collapse threshold')
ax.axhline(0.20, color='green', linestyle='--', alpha=0.5, label='Recovery threshold')
ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title(f'Collapse Branch: λ Sweep Down ({EPOCHS_PER_LAMBDA} ep/λ)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(LAMBDA_END - 0.02, LAMBDA_START + 0.02)

ax = axes[1]
df_mean = df.groupby('lambda')['error'].agg(['mean', 'std']).reset_index()
ax.fill_between(df_mean['lambda'], df_mean['mean'] - df_mean['std'], df_mean['mean'] + df_mean['std'], alpha=0.3, color='red')
ax.plot(df_mean['lambda'], df_mean['mean'], 'r-s', linewidth=2, markersize=5, label='Collapse branch (down)')
ax.axhline(0.40, color='red', linestyle='--', alpha=0.5, label='Collapse threshold')
ax.axhline(0.20, color='green', linestyle='--', alpha=0.5, label='Recovery threshold')
ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title(f'Mean ± Std ({len(results)} runs)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(LAMBDA_END - 0.02, LAMBDA_START + 0.02)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_sweep.png', dpi=150)
plt.show()

print(f'\n📈 Figure saved to {SAVE_DIR}/figures/')